<a href="https://colab.research.google.com/github/rkgupta7463/ML-projects-code/blob/main/Resume_Insight_Extaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install PyPDF2 python-docx spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184489 sha256=870249c88fd618a658710acb511efdc002512d70611d7509e54292f3bb39571b
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx
2023-07-28 03:56:54.665519: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 23.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
!pip install PyPDF2

In [13]:
import os
import json
import spacy
import PyPDF2
from docx import Document
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        pdf_reader = PyPDF2.PdfFileReader(file)
        text = ""
        for page_num in range(pdf_reader.numPages):
            page = pdf_reader.getPage(page_num)
            text += page.extract_text()
        return text

def extract_text_from_docx(docx_path):
    doc = Document(docx_path)
    paragraphs = [p.text for p in doc.paragraphs]
    return " ".join(paragraphs)

def parse_resume(text):
    doc = nlp(text)
    skills = []
    education = []
    work_experience = []

    # Custom patterns for education and work experience
    edu_patterns = [{"LOWER": {"IN": ["education", "qualification", "academic", "degree", "certification"]}}]
    exp_patterns = [{"LOWER": {"IN": ["work", "experience", "employment", "professional", "career"]}}]

    # Named Entity Recognition (NER) for skills
    for ent in doc.ents:
        if ent.label_ == "ORG" or ent.label_ == "PRODUCT":
            skills.append(ent.text)

    # Initialize Matcher and add patterns for education and work experience
    matcher = Matcher(nlp.vocab)
    matcher.add("Education", [edu_patterns])
    matcher.add("Experience", [exp_patterns])

    matches = matcher(doc)

    for match_id, start, end in matches:
        match_text = doc[start:end].text.strip()
        if doc.vocab.strings[match_id] == "Education":
            education.append(match_text)
        elif doc.vocab.strings[match_id] == "Experience":
            work_experience.append(match_text)

    return {"skills": skills, "education": education, "work_experience": work_experience}

def main():
    # Replace 'path/to/resume.pdf' with the path to the resume document.
    resume_path = "/content/resume.docx"
    ext = os.path.splitext(resume_path)[1]

    if ext == ".pdf":
        text = extract_text_from_pdf(resume_path)
    elif ext == ".docx":
        text = extract_text_from_docx(resume_path)
    else:
        raise ValueError("Unsupported file format. Only PDF and DOCX are supported.")

    parsed_info = parse_resume(text)
    print(json.dumps(parsed_info, indent=4))

if __name__ == "__main__":
    main()


{
    "skills": [
        "CSS & Bootstrap",
        "BIHAR BOARD",
        "BCA",
        "KL University",
        "EXPERIENCE Data",
        "Front-end",
        "PLATFORM",
        "CSS",
        "Web Developer Data Science & Machine Learning"
    ],
    "education": [
        "EDUCATION"
    ],
    "work_experience": [
        "work",
        "CAREER",
        "EXPERIENCE"
    ]
}
